In [1]:
import sys
sys.path.append('/home/users/rosealyd/ML_sat_obs/GCM_comp/')
import matplotlib.pyplot as plt
import numpy as np


In [32]:
hadgem_dict = np.load('/home/users/rosealyd/ML_sat_obs/GCM_comp/hadgem_data.npy').item()

In [47]:
import pickle
with open('/home/users/rosealyd/ML_sat_obs/ML_models/lightgbm_GCMcomp_cf_mse.sav', 'rb') as f:
        bst = pickle.load(f)

In [48]:
X_variables = ['sst', 'EIS', 'tot_aod','tot_ang', 'w500', 'RH850', 'upper_level_winds']

In [49]:
print(np.nanmin(hadgem_dict['w500']))
print(np.nanmax(hadgem_dict['w500']))
print(np.nanmin(hadgem_dict['tot_ang']))
print(np.nanmax(hadgem_dict['tot_ang']))

-0.9253468
0.7904648
0.08067308410532625
2.163249707922746


In [62]:
bad = (hadgem_dict['sst'][0] < 240)
hadgem_dict['sst'][0][bad] = np.nan

/opt/jaspy/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in less
  """Entry point for launching an IPython kernel.


In [ ]:

predicted_cfs = np.empty(hadgem_dict['sst'][0].shape)
xais = []
for month in range(1):
    for lat in range(len(hadgem_dict['sst'][month])):
            for lon in range(len(hadgem_dict['sst'][month][lat])):
                xai = []
                for var in X_variables:
                    xai.append(hadgem_dict[var][month][lat][lon])
                if (True not in np.isnan(xai)) and (hadgem_dict['RH850'][month][lat][lon] < 81):
                    predicted_cfs[lat][lon] = bst.predict([xai])[0]
                    xais.append(xai)
    xais = np.array(xais)
    pred_cfs = bst.predict(xais)


/opt/jaspy/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Warning: converting a masked element to nan.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
plt.hist(pred_cfs)
plt.show()

In [ ]:
plt.pcolormesh(predicted_cfs, vmin = 0, vmax = 1.)
plt.colorbar()
plt.show()

In [ ]:
plt.pcolormesh(hadgem_dict['cf'][0])
plt.colorbar()
plt.show()